In [1]:
from aux_func_V3 import *
from aux_gif2D import *
import API_Numpy
import API_TensorFlow

import dill
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
API=API_TensorFlow
# API=API_Numpy

ε=1e-40
WENOs_list=[
    'WENO-Z (BI)',
    'WENO-Z',
    'WENO-Zp',
    'WENO-Z (MS)',
    'WENO-Zp (MS)',
    'WENO-Zp (BI)',
    'WENO-JS (BI)',
    'WENO-ZC (MS)',
    'WENO-ZC (BI)'
    ]

In [3]:
γ = 7.0/5.0
N = 50
x0=1/6
x, y, Δx, Δy, U0 = CondiçãoInicialDoubleMach_N(N, γ,x0)
cfl = 0.2
t_final = 0.2

GhostPointsX=DoubleMachGhostPointsX(L=3,R=3,γ=γ)
GhostPointsY=DoubleMachGhostPointsY(L=3,R=3,γ=γ,x0=x0,x=x)

WENOs={}

for name in WENOs_list:
    WENOs[name]=WENO_dict[name+' 2D'](API,
                            euler_equation_2D,
                            γ=γ)

In [ ]:
for name in WENOs_list:
    create_data(
        'DoubleMach',
        N,
        name,
        U0,
        WENOs[name],
        0.001,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        NullForce,
        continue_flag=False)

    create_movie(
        'DoubleMach',
        N,
        name,
        x,
        y,
        figsize=(3*8,8),
        vmin=1,
        vmax=23,
        levels=100,
        xlim=(0,3),
        ylim=None)

In [3]:
γ = 7.0/5.0
θ = API_Numpy.pi/6.0
N = 100 # 400
x0=1/6
x, y, Δx, Δy, U0 = CondiçãoInicialShockEntropy2D_N(N, γ,θ)
cfl = 0.5
t_final = 1.8

GhostPointsX=ShockEntropy2DGhostPointsX(γ,Δx,y,θ)
GhostPointsY=ShockEntropy2DGhostPointsY()

WENOs={}

for name in WENOs_list:
    WENOs[name]=WENO_dict[name+' 2D'](API,
                            euler_equation_2D,
                            γ=γ)

In [4]:
for name in WENOs_list:
    create_data(
        'ShockEntropy',
        N,
        name,
        U0,
        WENOs[name],
        0.01,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        NullForce,
        continue_flag=False)

    create_movie(
        'ShockEntropy',
        N,
        name,
        x,
        y,
        figsize=(2*8,8),
        vmin=0.75,
        vmax=4.75,
        levels=15,
        xlim=None,
        ylim=None)

WENO-Z (BI)
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Z                           
Tempo inicial: 0.0


KeyboardInterrupt: 

In [2]:
def MalhaRetangular(N):
   Δx = 4.0/(4*N); Δy = 1/N
   x = API_Numpy.arange(0.0,4.0+Δx,Δx)
   y = API_Numpy.arange(0.0,1.0+Δy,Δy)

   X = API_Numpy.stack([x]*y.shape[0],axis=1)
   Y = API_Numpy.stack([y]*x.shape[0],axis=0)

   return X, Y, Δx, Δy

def CondiçãoInicialDoubleMach(X, Y, γ,x0):
   Reg1=X<x0+Y/np.sqrt(3)
   Reg2=X>=x0+Y/np.sqrt(3)

   R0 = API_Numpy.zeros(X.shape)
   P0 = API_Numpy.ones(X.shape)
   U0 = API_Numpy.zeros(X.shape)
   V0 = API_Numpy.zeros(X.shape)

   R0[Reg1]=8
   U0[Reg1]=8.25*np.sqrt(3)/2
   V0[Reg1]=-8.25/2
   P0[Reg1]=116.5

   R0[Reg2]=1.4

   Q0 = API_Numpy.stack([R0, R0*U0, R0*V0, P0/(γ-1)+R0*(U0**2+V0**2)/2],axis=0)
   return Q0

def CondiçãoInicialDoubleMach_N(N, γ,x0):
   X, Y, Δx, Δy = MalhaRetangular(N)
   Q0 = CondiçãoInicialDoubleMach(X, Y, γ,x0)
   return X, Y, Δx, Δy, Q0

def NullForce(Q,API):
   return 0.0

In [2]:
class DoubleMachGhostPointsX2:
    def __init__(self,L,R,γ):
        self.γ=γ
        self.L=L
        self.R=R
        self.uL=np.asarray([
            8,
            4*8.25*np.sqrt(3),
            -4*8.25,
            116.5/(γ - 1) + 4*(8.25)**2
            ])
    def __call__(self,Q,API,t=None):
        pre_shape=API.shape(Q)[:-3]
        uL_shape=API.cast(API.concat([pre_shape,(4,1,1)],axis=0),'int32')
        ones_shape=API.cast(API.concat([pre_shape,(1,self.L,API.shape(Q)[-1])],axis=0),'int32')
        UL=API.reshape(self.uL,uL_shape)*API.ones(ones_shape,dtype='float64')
        UR=API.flip(Q[...,-self.R-1:-1,:],axis=(-2,))

        Qg = [UL,Q,UR]

        return API.concat(Qg,-2)

class DoubleMachGhostPointsY2:
    def __init__(self,L,R,γ,x0,x):
        self.γ=γ
        self.L=L
        self.R=R
        self.x0=x0
        self.x=x[:,0]
        self.xL=x[:,0]<x0
        self.xR=x[:,0]>=x0
        self.uL=np.asarray([
            8,
            4*8.25*np.sqrt(3),
            -4*8.25,
            116.5/(γ - 1) + 4*(8.25)**2
            ])
        self.uR=np.asarray([
            1.4,
            0,
            0,
            1/(γ-1)
            ])
        self.Reflect=np.asarray([1,1,-1,1],dtype='float64') # indice da dimensão que será refletida
    def __call__(self,Q,API,t):
        Lu=API.shape(Q)[-3]
        Lx=API.shape(Q)[-2]
        pre_shape=API.shape(Q)[:-3]

        uLR_shape=API.cast(API.concat([pre_shape,(1,Lx,1)],axis=0),'int32')
        ones_L_shape=API.cast(API.concat([pre_shape,(Lu,1,self.L)],axis=0),'int32')
        ones_R_shape=API.cast(API.concat([pre_shape,(Lu,1,self.R)],axis=0),'int32')
        var_L_shape=API.cast(API.concat([pre_shape,(1,Lx,self.L)],axis=0),'int32')
        var_R_shape=API.cast(API.concat([pre_shape,(1,Lx,self.R)],axis=0),'int32')

        UL_condL=API.cast(API.reshape(self.xL,uLR_shape),'float64')
        UL_condL=API.cast(UL_condL*API.ones(ones_L_shape,dtype='float64'),'bool')
        UL_condR=API.cast(API.reshape(self.xR,uLR_shape),'float64')
        UL_condR=API.cast(UL_condR*API.ones(ones_L_shape,dtype='float64'),'bool')

        reflect_shape=API.cast(API.concat([pre_shape,(4,1,1)],axis=0),'int32')
        reflect_value=API.reshape(self.Reflect,reflect_shape)*API.ones(var_L_shape,dtype='float64')
        
        UL=API.reshape(self.uL,reflect_shape)*API.ones(var_L_shape,dtype='float64')
        UL=API.where(
            UL_condL,
            UL,
            reflect_value*API.flip(Q[...,:,1:self.L+1],axis=(-1,))
            )

        s=self.x0 + (1 + 20*t)/np.sqrt(3)
        xR=self.x<s
        UR_condL=API.cast(API.reshape(xR,uLR_shape),'float64')
        UR_condL=API.cast(UR_condL*API.ones(ones_R_shape,dtype='float64'),'bool')
        #xR=self.x>=s
        UR=API.where(
            UR_condL,
            API.reshape(self.uL,reflect_shape)*API.ones(var_R_shape,dtype='float64'),
            API.reshape(self.uR,reflect_shape)*API.ones(var_R_shape,dtype='float64'))

        Qg = [UL,Q,UR]

        return API.concat(Qg,-1)